# Multi-Agent Systems and Agentic Patterns with LlamaIndex

## Tutorial Overview

In this comprehensive tutorial, we'll explore **multi-agent systems** and **agentic patterns** using **LlamaIndex** with **OpenAI's gpt-4o-mini** model as our LLM backend.

---

## What is LlamaIndex?

**LlamaIndex** is a powerful data framework designed to help you build context-augmented LLM applications. It provides:

- **Data Connectors**: Ingest data from various sources (APIs, PDFs, databases, etc.)
- **Data Indexes**: Structure your data for efficient retrieval
- **Agents**: Intelligent decision-making components that can use tools and reason about tasks
- **Query Engines**: Retrieve relevant context and generate answers
- **Chat Engines**: Build conversational interfaces

---

## Why are Agentic Patterns Important?

**Agentic patterns** enable AI systems to:

1. **Break down complex problems** into manageable sub-tasks
2. **Make autonomous decisions** about which tools or approaches to use
3. **Coordinate multiple specialized agents** for better outcomes
4. **Self-improve** through feedback loops and evaluation
5. **Scale reasoning** beyond single-shot prompting

In production systems, these patterns help create more robust, maintainable, and powerful AI applications.

---

## Patterns We'll Cover

1. **Prompt Chaining**: Sequential agent communication
2. **Routing**: Intelligent task delegation
3. **Parallelization**: Concurrent execution and aggregation
4. **Orchestrator-Worker**: Hierarchical task management
5. **Evaluator-Optimizer**: Iterative improvement through feedback

Let's get started! 🚀

---

## 1. Setup & Installation

First, let's install the required dependencies:

In [1]:
# Install required packages
!pip install llama-index llama-index-llms-openai openai python-dotenv -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.9/11.9 MB 98.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 303.3/303.3 kB 26.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.8/51.8 kB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 96.8/96.8 kB 9.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.9/63.9 kB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 329.1/329.1 kB 26.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 65.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.0/88.0 kB 9.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.0/51.0 kB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 150.7/150.7 kB 17.1 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
ipython 7.34.0 requires jedi>=0.16, 

### Configure OpenAI API Key

You'll need an OpenAI API key. Get one from: https://platform.openai.com/api-keys

In [2]:
import getpass
import os

if "OPENAI_API_KEY" not in os.environ:
    os.environ["OPENAI_API_KEY"] = getpass.getpass("Enter your OpenAI API Key: ")

if "OPENAI_BASE_URL" not in os.environ:
    os.environ["OPENAI_BASE_URL"] = getpass.getpass("Enter your OpenAI Base URL: ")

Enter your OpenAI API Key: ··········
Enter your OpenAI Base URL: ··········


---

## 2. Model Configuration

Let's set up our OpenAI configuration using the **gpt-4o-mini** model. We'll create a reusable setup that we can use throughout the tutorial.

In [4]:
from llama_index.llms.openai import OpenAI
from llama_index.core import Settings

# Configure OpenAI
llm = OpenAI(
    model="gpt-4.1-mini",
    api_key=os.getenv("OPENAI_API_KEY"),
    api_base=os.getenv("OPENAI_BASE_URL"),
    temperature=0.7,
    max_tokens=2048,
)

# Set global defaults for LlamaIndex
Settings.llm = llm
Settings.chunk_size = 512

print("✅ Model configuration complete!")
print(f"📋 LLM: gpt-4.1-mini (via OpenAI)")
print(f"📋 API Base: {os.getenv('OPENAI_BASE_URL')}")

✅ Model configuration complete!
📋 LLM: gpt-4.1-mini (via OpenAI)
📋 API Base: https://ds-ai-internship.openai.azure.com/openai/v1/


In [5]:
# Test the model configuration
response = llm.complete("Hello! Can you confirm you're working? Reply with just 'Yes, I'm ready!'")
print(f"Model Response: {response.text}")

Model Response: Yes, I'm ready!


---

## 3. Agentic Patterns

Now let's explore each pattern with practical examples!

---

### Pattern A: Prompt Chaining

**Concept**: The output of one agent becomes the input of another agent. This creates a sequential pipeline where each step builds on the previous one.

**Use Cases**:
- Content creation pipelines (outline → draft → polish)
- Data transformation workflows
- Multi-step reasoning tasks

**Implementation**: We'll create Agent A (Story Generator) and Agent B (Story Improver)

In [6]:
from llama_index.core.llms import ChatMessage

class PromptChainExample:
    """Demonstrates prompt chaining with two sequential agents."""

    def __init__(self, llm):
        self.llm = llm

    def agent_a_generate_story(self, topic: str) -> str:
        """
        Agent A: Generates a basic story outline.
        """
        print("\n🤖 Agent A (Story Generator) is working...")

        prompt = f"""You are a creative story generator.
        Create a brief, simple story (3-4 sentences) about: {topic}
        Keep it concise and straightforward."""

        messages = [ChatMessage(role="user", content=prompt)]
        response = self.llm.chat(messages)

        story = response.message.content
        print(f"\n📝 Agent A Output:\n{story}")
        return story

    def agent_b_improve_story(self, story: str) -> str:
        """
        Agent B: Takes the story from Agent A and improves it.
        """
        print("\n\n🤖 Agent B (Story Improver) is working...")

        prompt = f"""You are a story editor. Take this story and enhance it by:
        1. Adding more vivid descriptions
        2. Improving the narrative flow
        3. Making it more engaging

        Original Story:
        {story}

        Provide the improved version:"""

        messages = [ChatMessage(role="user", content=prompt)]
        response = self.llm.chat(messages)

        improved_story = response.message.content
        print(f"\n✨ Agent B Output:\n{improved_story}")
        return improved_story

    def run_chain(self, topic: str):
        """
        Execute the full prompt chain: Agent A → Agent B
        """
        print("="*60)
        print("🔗 PROMPT CHAINING EXAMPLE")
        print("="*60)

        # Step 1: Agent A generates the story
        story = self.agent_a_generate_story(topic)

        # Step 2: Agent B improves the story (chaining the output)
        improved_story = self.agent_b_improve_story(story)

        print("\n" + "="*60)
        print("✅ Prompt Chain Complete!")
        print("="*60)

        return improved_story

# Run the example
chain_example = PromptChainExample(llm)
final_story = chain_example.run_chain("a robot learning to paint")

🔗 PROMPT CHAINING EXAMPLE

🤖 Agent A (Story Generator) is working...

📝 Agent A Output:
A small robot named Pixel watched colorful paintings in a gallery and wanted to create art too. It picked up a brush and began mixing bright colors on a canvas. At first, its strokes were shaky, but with practice, Pixel painted a beautiful sunset. Proud of its work, the robot smiled, knowing it had learned to express itself through painting.


🤖 Agent B (Story Improver) is working...

✨ Agent B Output:
In the heart of a bustling gallery filled with vibrant masterpieces, a small robot named Pixel stood quietly, its optical sensors gleaming with wonder. The walls were alive with splashes of crimson, gold, and indigo—each painting whispering stories through swirling brushstrokes and radiant hues. Mesmerized, Pixel felt a stirring deep within its circuits, a yearning to create its own symphony of color.

With determination, Pixel gently picked up a slender brush, its metal fingers surprisingly delicate.

---

### Pattern B: Routing

**Concept**: A router agent analyzes the input and decides which specialized agent or tool should handle the request.

**Use Cases**:
- Customer service bots (route to departments)
- Multi-domain question answering
- Tool selection in complex systems

**Implementation**: We'll create a router that directs queries to specialized agents (Math, History, or General)

In [7]:
from llama_index.core.agent import ReActAgent
from llama_index.core.tools import FunctionTool
import json

class RoutingExample:
    """Demonstrates routing pattern with specialized agents."""

    def __init__(self, llm):
        self.llm = llm

    def math_specialist(self, query: str) -> str:
        """Specialized agent for mathematical queries."""
        prompt = f"You are a math expert. Answer this mathematical question concisely: {query}"
        messages = [ChatMessage(role="user", content=prompt)]
        response = self.llm.chat(messages)
        return f"🔢 Math Specialist: {response.message.content}"

    def history_specialist(self, query: str) -> str:
        """Specialized agent for historical queries."""
        prompt = f"You are a history expert. Answer this historical question concisely: {query}"
        messages = [ChatMessage(role="user", content=prompt)]
        response = self.llm.chat(messages)
        return f"📚 History Specialist: {response.message.content}"

    def general_specialist(self, query: str) -> str:
        """General purpose agent for other queries."""
        prompt = f"Answer this question: {query}"
        messages = [ChatMessage(role="user", content=prompt)]
        response = self.llm.chat(messages)
        return f"💡 General Specialist: {response.message.content}"

    def route_query(self, query: str) -> str:
        """
        Router agent that determines which specialist should handle the query.
        """
        print("\n🔀 Router Agent analyzing query...")

        # Router decides which agent to use
        routing_prompt = f"""Analyze this query and determine the best specialist:

        Query: {query}

        Options:
        - 'math': For mathematical calculations, equations, or number problems
        - 'history': For historical events, dates, or historical figures
        - 'general': For everything else

        Respond with ONLY one word: math, history, or general"""

        messages = [ChatMessage(role="user", content=routing_prompt)]
        response = self.llm.chat(messages)
        route_decision = response.message.content.strip().lower()

        print(f"🎯 Router Decision: Route to '{route_decision}' specialist\n")

        # Route to appropriate specialist
        if 'math' in route_decision:
            return self.math_specialist(query)
        elif 'history' in route_decision:
            return self.history_specialist(query)
        else:
            return self.general_specialist(query)

    def demo(self):
        """Demonstrate routing with different types of queries."""
        print("="*60)
        print("🔀 ROUTING PATTERN EXAMPLE")
        print("="*60)

        queries = [
            "What is the square root of 144?",
            "When did World War II end?",
            "What is the best way to learn programming?"
        ]

        for query in queries:
            print(f"\n📥 Query: {query}")
            result = self.route_query(query)
            print(f"📤 {result}")
            print("-" * 60)

        print("\n" + "="*60)
        print("✅ Routing Demo Complete!")
        print("="*60)

# Run the example
routing_example = RoutingExample(llm)
routing_example.demo()

🔀 ROUTING PATTERN EXAMPLE

📥 Query: What is the square root of 144?

🔀 Router Agent analyzing query...
🎯 Router Decision: Route to 'math' specialist

📤 🔢 Math Specialist: The square root of 144 is 12.
------------------------------------------------------------

📥 Query: When did World War II end?

🔀 Router Agent analyzing query...
🎯 Router Decision: Route to 'history' specialist

📤 📚 History Specialist: World War II ended in 1945. In Europe, it concluded with Germany's surrender on May 8, 1945 (V-E Day), and in the Pacific, it ended with Japan's surrender on September 2, 1945 (V-J Day).
------------------------------------------------------------

📥 Query: What is the best way to learn programming?

🔀 Router Agent analyzing query...
🎯 Router Decision: Route to 'general' specialist

📤 💡 General Specialist: The best way to learn programming depends on your goals, learning style, and resources, but here are some widely effective strategies:

1. **Start with the Basics:** Choose a beginne

---

### Pattern C: Parallelization

**Concept**: Run multiple agents or tasks simultaneously and aggregate their results.

**Use Cases**:
- Multi-perspective analysis
- Concurrent research tasks
- Ensemble approaches

**Implementation**: We'll analyze a topic from multiple perspectives in parallel, then synthesize the results

In [8]:
from concurrent.futures import ThreadPoolExecutor, as_completed
from typing import List
import time

class ParallelizationExample:
    """Demonstrates parallel execution of multiple agents using threading."""

    def __init__(self, llm):
        self.llm = llm

    def analyze_from_perspective(self, topic: str, perspective: str) -> str:
        """
        Analyze a topic from a specific perspective.
        """
        print(f"🔄 Analyzing from {perspective} perspective...")

        prompt = f"""Analyze the following topic from a {perspective} perspective.
        Provide 2-3 key insights.

        Topic: {topic}

        Perspective: {perspective}
        """

        messages = [ChatMessage(role="user", content=prompt)]
        response = self.llm.chat(messages)  # Synchronous call

        result = f"**{perspective.upper()} PERSPECTIVE:**\n{response.message.content}"
        print(f"✅ {perspective} analysis complete")
        return result

    def parallel_analyze(self, topic: str, perspectives: List[str]) -> List[str]:
        """
        Run multiple analyses in parallel using ThreadPoolExecutor.
        """
        print(f"\n🚀 Starting parallel analysis of: {topic}")
        print(f"📋 Perspectives: {', '.join(perspectives)}\n")

        results = []

        # Use ThreadPoolExecutor for parallel execution
        with ThreadPoolExecutor(max_workers=len(perspectives)) as executor:
            # Submit all tasks
            future_to_perspective = {
                executor.submit(self.analyze_from_perspective, topic, perspective): perspective
                for perspective in perspectives
            }

            # Collect results as they complete
            for future in as_completed(future_to_perspective):
                try:
                    result = future.result()
                    results.append(result)
                except Exception as e:
                    perspective = future_to_perspective[future]
                    print(f"❌ Error analyzing {perspective} perspective: {e}")

        return results

    def synthesize_results(self, results: List[str]) -> str:
        """
        Aggregate and synthesize the parallel results.
        """
        print("\n🔗 Synthesizing results...")

        combined_insights = "\n\n".join(results)

        synthesis_prompt = f"""You have received multiple perspectives on a topic.
        Synthesize these into a brief, coherent summary that captures the key themes.

        Perspectives:
        {combined_insights}

        Provide a 2-3 sentence synthesis:"""

        messages = [ChatMessage(role="user", content=synthesis_prompt)]
        response = self.llm.chat(messages)

        return response.message.content

    def run_demo(self):
        """Demonstrate parallel execution."""
        print("="*60)
        print("⚡ PARALLELIZATION PATTERN EXAMPLE")
        print("="*60)

        topic = "Artificial Intelligence in Healthcare"
        perspectives = ["technical", "ethical", "economic"]

        # Track time for demonstration
        start_time = time.time()

        # Run parallel analyses
        results = self.parallel_analyze(topic, perspectives)

        elapsed_time = time.time() - start_time

        # Display individual results
        print("\n" + "="*60)
        print("📊 INDIVIDUAL PERSPECTIVES:")
        print("="*60)
        for result in results:
            print(f"\n{result}")

        # Synthesize
        synthesis = self.synthesize_results(results)

        print("\n" + "="*60)
        print("🎯 SYNTHESIZED SUMMARY:")
        print("="*60)
        print(f"\n{synthesis}")

        print("\n" + "="*60)
        print(f"✅ Parallelization Demo Complete! (Executed in {elapsed_time:.2f}s)")
        print("="*60)

# Run the example
parallel_example = ParallelizationExample(llm)
parallel_example.run_demo()

⚡ PARALLELIZATION PATTERN EXAMPLE

🚀 Starting parallel analysis of: Artificial Intelligence in Healthcare
📋 Perspectives: technical, ethical, economic

🔄 Analyzing from technical perspective...
🔄 Analyzing from ethical perspective...
🔄 Analyzing from economic perspective...
✅ ethical analysis complete
✅ economic analysis complete
✅ technical analysis complete

📊 INDIVIDUAL PERSPECTIVES:

**ETHICAL PERSPECTIVE:**
Analyzing Artificial Intelligence (AI) in Healthcare from an ethical perspective reveals several important considerations:

1. **Patient Privacy and Data Security:**  
   AI systems in healthcare often rely on large volumes of sensitive patient data. Ethically, it is crucial to ensure that this data is collected, stored, and processed with strict confidentiality and robust security measures to prevent breaches. Patients must give informed consent, understanding how their data will be used, and there should be transparency about AI’s role in diagnosis or treatment.

2. **Bias an

---

### Pattern D: Orchestrator-Worker

**Concept**: A manager/orchestrator agent breaks down complex tasks and assigns sub-tasks to worker agents.

**Use Cases**:
- Project planning and execution
- Complex research tasks
- Multi-step content generation

**Implementation**: An orchestrator plans a blog post, then delegates writing tasks to worker agents

In [9]:
from typing import List, Dict
import json

class OrchestratorWorkerExample:
    """Demonstrates orchestrator-worker pattern."""

    def __init__(self, llm):
        self.llm = llm

    def orchestrator_plan(self, task: str) -> List[Dict[str, str]]:
        """
        Orchestrator Agent: Breaks down a complex task into sub-tasks.
        """
        print("\n🎯 Orchestrator Agent: Planning task breakdown...")

        planning_prompt = f"""You are a project orchestrator. Break down this task into 3 clear sub-tasks.

        Main Task: {task}

        Return your response in this exact JSON format:
        {{
            "subtasks": [
                {{"id": 1, "description": "First sub-task"}},
                {{"id": 2, "description": "Second sub-task"}},
                {{"id": 3, "description": "Third sub-task"}}
            ]
        }}

        Provide ONLY the JSON, no other text."""

        messages = [ChatMessage(role="user", content=planning_prompt)]
        response = self.llm.chat(messages)

        try:
            # Extract JSON from response
            response_text = response.message.content.strip()
            # Remove markdown code blocks if present
            if response_text.startswith("```"):
                response_text = response_text.split("```")[1]
                if response_text.startswith("json"):
                    response_text = response_text[4:]

            plan = json.loads(response_text)
            subtasks = plan["subtasks"]

            print("\n📋 Orchestrator's Plan:")
            for subtask in subtasks:
                print(f"  {subtask['id']}. {subtask['description']}")

            return subtasks
        except Exception as e:
            print(f"Error parsing plan: {e}")
            # Fallback to manual subtasks
            return [
                {"id": 1, "description": f"Research and outline for: {task}"},
                {"id": 2, "description": f"Write main content for: {task}"},
                {"id": 3, "description": f"Review and finalize: {task}"}
            ]

    def worker_execute(self, subtask: Dict[str, str]) -> str:
        """
        Worker Agent: Executes a specific sub-task.
        """
        subtask_id = subtask['id']
        description = subtask['description']

        print(f"\n👷 Worker Agent {subtask_id}: Executing sub-task...")

        execution_prompt = f"""You are a worker agent. Complete this specific sub-task concisely (2-3 sentences):

        Sub-task: {description}

        Provide your output:"""

        messages = [ChatMessage(role="user", content=execution_prompt)]
        response = self.llm.chat(messages)

        result = response.message.content
        print(f"✅ Worker {subtask_id} completed")

        return result

    def orchestrator_aggregate(self, task: str, results: List[str]) -> str:
        """
        Orchestrator Agent: Aggregates worker results into final output.
        """
        print("\n🎯 Orchestrator Agent: Aggregating results...")

        combined_work = "\n\n".join([f"Part {i+1}:\n{result}" for i, result in enumerate(results)])

        aggregation_prompt = f"""You are the orchestrator. Combine these worker outputs into a cohesive final result for the task: {task}

        Worker Outputs:
        {combined_work}

        Create a unified, polished final output:"""

        messages = [ChatMessage(role="user", content=aggregation_prompt)]
        response = self.llm.chat(messages)

        return response.message.content

    def run_demo(self, task: str):
        """Demonstrate the orchestrator-worker pattern."""
        print("="*60)
        print("👔 ORCHESTRATOR-WORKER PATTERN EXAMPLE")
        print("="*60)
        print(f"\n📌 Main Task: {task}")

        # Step 1: Orchestrator creates plan
        subtasks = self.orchestrator_plan(task)

        # Step 2: Workers execute sub-tasks
        results = []
        for subtask in subtasks:
            result = self.worker_execute(subtask)
            results.append(result)

        # Step 3: Orchestrator aggregates results
        final_output = self.orchestrator_aggregate(task, results)

        print("\n" + "="*60)
        print("🎉 FINAL OUTPUT:")
        print("="*60)
        print(f"\n{final_output}")

        print("\n" + "="*60)
        print("✅ Orchestrator-Worker Demo Complete!")
        print("="*60)

        return final_output

# Run the example
orchestrator_example = OrchestratorWorkerExample(llm)
final_result = orchestrator_example.run_demo("Write a short blog post about the benefits of meditation")

👔 ORCHESTRATOR-WORKER PATTERN EXAMPLE

📌 Main Task: Write a short blog post about the benefits of meditation

🎯 Orchestrator Agent: Planning task breakdown...

📋 Orchestrator's Plan:
  1. Research and gather information on the benefits of meditation
  2. Write a concise and engaging draft of the blog post
  3. Edit and finalize the blog post for clarity and flow

👷 Worker Agent 1: Executing sub-task...
✅ Worker 1 completed

👷 Worker Agent 2: Executing sub-task...
✅ Worker 2 completed

👷 Worker Agent 3: Executing sub-task...
✅ Worker 3 completed

🎯 Orchestrator Agent: Aggregating results...

🎉 FINAL OUTPUT:

Meditation offers numerous benefits that can significantly enhance your quality of life. By incorporating this simple practice into your daily routine, you can reduce stress, improve emotional health, and boost focus and concentration. Additionally, meditation promotes better sleep, lowers blood pressure, and increases self-awareness, contributing to overall well-being.

Discover ho

---

### Pattern E: Evaluator-Optimizer

**Concept**: Implement a feedback loop where one agent generates solutions and another evaluates and provides feedback for improvement.

**Use Cases**:
- Code review and refinement
- Content quality improvement
- Iterative problem-solving

**Implementation**: A generator creates code, an evaluator critiques it, and the generator improves based on feedback

In [10]:
class EvaluatorOptimizerExample:
    """Demonstrates evaluator-optimizer pattern with feedback loops."""

    def __init__(self, llm, max_iterations: int = 2):
        self.llm = llm
        self.max_iterations = max_iterations

    def generator_agent(self, task: str, feedback: str = None) -> str:
        """
        Generator Agent: Creates a solution (with optional feedback incorporation).
        """
        if feedback:
            print("\n🔧 Generator Agent: Improving solution based on feedback...")
            prompt = f"""You previously worked on this task: {task}

            You received this feedback:
            {feedback}

            Please create an improved version that addresses the feedback."""
        else:
            print("\n✍️ Generator Agent: Creating initial solution...")
            prompt = f"""Create a solution for this task: {task}

            Provide a concise solution (3-4 sentences)."""

        messages = [ChatMessage(role="user", content=prompt)]
        response = self.llm.chat(messages)

        solution = response.message.content
        print(f"\n📄 Generated Solution:\n{solution}")

        return solution

    def evaluator_agent(self, task: str, solution: str) -> Dict[str, any]:
        """
        Evaluator Agent: Critiques the solution and provides feedback.
        """
        print("\n🔍 Evaluator Agent: Analyzing solution...")

        evaluation_prompt = f"""You are a critical evaluator. Assess this solution:

        Task: {task}

        Solution:
        {solution}

        Provide:
        1. A quality score (1-10)
        2. Specific feedback for improvement (if score < 9)
        3. Whether it's acceptable (yes/no)

        Format:
        Score: [number]
        Acceptable: [yes/no]
        Feedback: [your feedback or 'None' if perfect]
        """

        messages = [ChatMessage(role="user", content=evaluation_prompt)]
        response = self.llm.chat(messages)

        evaluation_text = response.message.content
        print(f"\n📊 Evaluation:\n{evaluation_text}")

        # Parse evaluation (simple heuristic)
        lines = evaluation_text.lower().split('\n')

        acceptable = any('acceptable: yes' in line for line in lines)

        # Extract feedback
        feedback_lines = []
        capture = False
        for line in evaluation_text.split('\n'):
            if 'feedback:' in line.lower():
                capture = True
                feedback_lines.append(line.split(':', 1)[1].strip() if ':' in line else '')
            elif capture:
                feedback_lines.append(line)

        feedback = ' '.join(feedback_lines).strip()
        if not feedback or 'none' in feedback.lower():
            feedback = None

        return {
            'acceptable': acceptable,
            'feedback': feedback,
            'evaluation_text': evaluation_text
        }

    def run_feedback_loop(self, task: str):
        """
        Run the evaluator-optimizer feedback loop.
        """
        print("="*60)
        print("🔁 EVALUATOR-OPTIMIZER PATTERN EXAMPLE")
        print("="*60)
        print(f"\n📌 Task: {task}")

        solution = None
        feedback = None

        for iteration in range(self.max_iterations):
            print(f"\n{'='*60}")
            print(f"🔄 ITERATION {iteration + 1}")
            print(f"{'='*60}")

            # Generate (or improve) solution
            solution = self.generator_agent(task, feedback)

            # Evaluate solution
            evaluation = self.evaluator_agent(task, solution)

            # Check if acceptable
            if evaluation['acceptable']:
                print("\n✅ Solution accepted!")
                break
            else:
                print("\n⚠️ Solution needs improvement")
                feedback = evaluation['feedback']
                if iteration < self.max_iterations - 1:
                    print(f"\n📝 Feedback for next iteration: {feedback}")

        print("\n" + "="*60)
        print("🎉 FINAL SOLUTION:")
        print("="*60)
        print(f"\n{solution}")

        print("\n" + "="*60)
        print("✅ Evaluator-Optimizer Demo Complete!")
        print("="*60)

        return solution

# Run the example
evaluator_example = EvaluatorOptimizerExample(llm, max_iterations=2)
final_solution = evaluator_example.run_feedback_loop(
    "Write a Python function that checks if a number is prime"
)

🔁 EVALUATOR-OPTIMIZER PATTERN EXAMPLE

📌 Task: Write a Python function that checks if a number is prime

🔄 ITERATION 1

✍️ Generator Agent: Creating initial solution...

📄 Generated Solution:
Here's a concise Python function to check if a number is prime. It first handles edge cases (numbers less than 2), then checks divisibility from 2 up to the square root of the number for efficiency:

```python
def is_prime(n):
    if n < 2:
        return False
    for i in range(2, int(n**0.5) + 1):
        if n % i == 0:
            return False
    return True
```

🔍 Evaluator Agent: Analyzing solution...

📊 Evaluation:
Score: 9  
Acceptable: yes  
Feedback: The solution is correct, efficient, and concise. For further improvement, consider adding a docstring to explain the function's purpose and inputs, and possibly handle non-integer inputs gracefully or document that the input should be an integer. Adding type hints could also enhance readability.

✅ Solution accepted!

🎉 FINAL SOLUTION:

Her

---

## 4. Conclusion & Pattern Selection Guide

### When to Use Each Pattern

| Pattern | Best For | Key Benefit | Complexity |
|---------|----------|-------------|------------|
| **Prompt Chaining** | Sequential pipelines, data transformation | Simple, predictable flow | ⭐ Low |
| **Routing** | Multi-domain systems, request classification | Specialization, efficiency | ⭐⭐ Medium |
| **Parallelization** | Independent tasks, time-sensitive operations | Speed, diverse perspectives | ⭐⭐ Medium |
| **Orchestrator-Worker** | Complex projects, hierarchical tasks | Organization, scalability | ⭐⭐⭐ High |
| **Evaluator-Optimizer** | Quality-critical outputs, iterative refinement | Quality, self-improvement | ⭐⭐⭐ High |

### Pattern Combinations

These patterns can be **combined** for more sophisticated systems:

- **Orchestrator + Routing**: Manager delegates to specialized workers
- **Parallelization + Evaluator**: Multiple solutions generated, best one selected
- **Chaining + Optimizer**: Each step in chain has evaluation/refinement

### Best Practices

1. **Start Simple**: Begin with prompt chaining, add complexity as needed
2. **Monitor Costs**: Even with free models, be mindful of rate limits and quotas
3. **Error Handling**: Implement retries and fallbacks for production systems
4. **Logging**: Track agent decisions and outputs for debugging
5. **Human in the Loop**: Add checkpoints for critical decisions

### Next Steps

- Explore LlamaIndex's built-in agent frameworks
- Add RAG (Retrieval-Augmented Generation) to your agents
- Implement tool use with agents
- Build custom workflows with `llama_index.core.workflow`
- Try other OpenRouter models as you scale

### Resources

- [LlamaIndex Documentation](https://docs.llamaindex.ai/)
- [LlamaIndex Agent Guide](https://docs.llamaindex.ai/en/stable/understanding/agent/)
---

**Happy Building! 🚀**